In [1]:
%cd ..

/mnt/nfs/clustersw/Debian/bookworm/jupyterhub/1.0/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/nfs/scistore16/edelgrp/fzimin/depth-poset


In [2]:
import os

import numpy as np
import pandas as pd
import pickle as pkl

#from tqdm.notebook import tqdm
from tqdm import tqdm

# Load and Format the Data

In [3]:
# define file paths
directory = "results/scores-on-random-alpha-complexes"
paths = np.sort([f'{directory}/{f}' for f in os.listdir(directory)])

# load them
ls = []
for path in tqdm(paths, total=len(paths)):
    with open(path, 'rb') as file:
        ls.append(pkl.load(file))

100%|██████████| 187/187 [00:00<00:00, 498.33it/s]


In [4]:
df_poset_scores = [pd.DataFrame(i['poset scores']) for i in ls]
for i in range(len(df_poset_scores)):
    df_poset_scores[i].insert(0, 'index', value=i)
df_poset_scores = pd.concat(df_poset_scores)

print(f'df_poset_scores.shape = {df_poset_scores.shape}')
df_poset_scores.head()

df_poset_scores.shape = (527, 9)


,index,object,number_of_nodes,number_of_minimal_nodes,number_of_maximal_nodes,height,width,minimum_maximal_chain,avarage_maximal_chain
0,0,full,63,51,42,3,51,1,1.688525
1,0,subposet dim=0,23,13,6,3,13,1,2.652174
2,0,subposet dim=1,40,38,36,1,38,1,1.105263
0,1,full,190,174,154,6,174,1,1.357955
1,1,subposet dim=0,23,14,4,6,14,1,3.533333


In [5]:
df_node_scores = [pd.DataFrame(i['node scores']) for i in ls]
for i in range(len(df_node_scores)):
    df_node_scores[i].insert(0, 'index', value=i)
df_node_scores = pd.concat(df_node_scores)

print(f'df_node_scores.shape = {df_node_scores.shape}')
df_node_scores.head()

df_node_scores.shape = (5734, 10)


,index,object,node,incomparable_number,incestors_number,incestors_height,incestors_width,successors_number,successors_height,successors_width
0,0,full,"ShallowPair([0.0000, 0.0133], dim=0)",58,0,0,0,4,1,3
1,0,subposet dim=0,"ShallowPair([0.0000, 0.0133], dim=0)",18,0,0,0,4,1,3
2,0,full,"ShallowPair([0.0000, 0.0011], dim=0)",59,2,1,1,1,0,1
3,0,subposet dim=0,"ShallowPair([0.0000, 0.0011], dim=0)",19,2,1,1,1,0,1
4,0,full,"ShallowPair([0.0000, 0.0150], dim=0)",52,0,0,0,10,1,6


In [6]:
df_node_scores[['index', 'object', 'incomparable_number', 'incestors_number',
                'incestors_height', 'incestors_width', 'successors_number',
                'successors_height', 'successors_width']].groupby(['index', 'object']).agg(['min', 'max'])

incomparable_number      incestors_number      \
                                     min  max              min max   
index object                                                         
0     full                            52   62                0   4   
      subposet dim=0                  12   21                0   4   
      subposet dim=1                  38   39                0   1   
1     full                           176  189                0   2   
      subposet dim=0                   9   22                0   2   
...                                  ...  ...              ...  ..   
184   subposet dim=0                   3    8                0   2   
185   full                             5    8                0   2   
      subposet dim=0                   5    8                0   2   
186   full                             0    8                0   4   
      subposet dim=0                   0    8                0   4   

                     incestors_height     incestors_width      \
                                  min max             min max   
index object                                                    
0     full                          0   1               0   3   
      subposet dim=0                0   1               0   3   
      subposet dim=1                0   0               0   1   
1     full                          0   1               0   1   
      subposet dim=0                0   1               0   1   
...                               ...  ..             ...  ..   
184   subposet dim=0                0   1               0   1   
185   full                          0   1               0   1   
      subposet dim=0                0   1               0   1   
186   full                          0   3               0   1   
      subposet dim=0                0   3               0   1   

                     successors_number     successors_height      \
                                   min max               min max   
index object                                                       
0     full                           0  10                 0   1   
      subposet dim=0                 0  10                 0   1   
      subposet dim=1                 0   0                 0   0   
1     full                           0  12                 0   4   
      subposet dim=0                 0  12                 0   4   
...                                ...  ..               ...  ..   
184   subposet dim=0                 0   7                 0   1   
185   full                           0   5                 0   1   
      subposet dim=0                 0   5                 0   1   
186   full                           0  10                 0   3   
      subposet dim=0                 0  10                 0   3   

                     successors_width      
                                  min max  
index object                               
0     full                          0   6  
      subposet dim=0                0   6  
      subposet dim=1                0   0  
1     full                          0   7  
      subposet dim=0                0   7  
...                               ...  ..  
184   subposet dim=0                0   4  
185   full                          0   3  
      subposet dim=0                0   3  
186   full                          0   5  
      subposet dim=0                0   5  

[510 rows x 14 columns]

In [7]:
pd.DataFrame(ls).groupby(['n', 'dim']).count()

points  stree  depth poset  poset scores  node scores
n  dim                                                       
12 1        25     25           25            25           25
   2        10     10           10            10           10
   3        10     10           10            10           10
   4        10     10           10            10           10
20 1        10     10           10            10           10
   2        10     10           10            10           10
   3        10     10           10            10           10
   4         5      5            5             5            5
24 1         1      1            1             1            1
   2         1      1            1             1            1
   3         1      1            1             1            1
   4         1      1            1             1            1
28 1        10     10           10            10           10
   2        10     10           10            10           10
   3         9      9            9             9            9
36 1        10     10           10            10           10
   2        10     10           10            10           10
44 1        10     10           10            10           10
   2         4      4            4             4            4
68 1        10     10           10            10           10
76 1        10     10           10            10           10
84 1        10     10           10            10           10